# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,22.53,96,98,2.35,EC,1715414179
1,1,shizunai-furukawacho,42.3339,142.3669,13.08,71,72,4.42,JP,1715414179
2,2,koryazhma,61.3143,47.1691,-0.05,79,100,0.40,RU,1715414180
3,3,hotan,37.0997,79.9269,29.55,6,100,3.40,CN,1715414180
4,4,wailua homesteads,22.0669,-159.3780,19.30,88,28,0.89,US,1715414181


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points('Lng',
                                           'Lat',
                                           geo=True,
                                           tiles='OSM',
                                           size='Humidity',
                                           frame_width=800,
                                           frame_height=600,
                                           color='City',
                                           alpha=0.5,
                                           xlabel='x',
                                           ylabel='y')

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_weather_df = city_data_df.loc[(city_data_df['Max Temp'] >= 21) &
                                      (city_data_df['Max Temp'] <= 30) &
                                      (city_data_df['Cloudiness'] < 20) &
                                      (city_data_df['Wind Speed'] < 2)]

# Drop any rows with null values
perfect_weather_df = perfect_weather_df.dropna(how='any')

# Display sample data
perfect_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,67,benguela,-12.5763,13.4055,28.07,74,12,1.39,AO,1715414207
89,89,namibe,-15.1961,12.1522,24.09,71,0,1.25,AO,1715414215
100,100,ardakan,32.3100,54.0175,26.28,17,6,1.38,IR,1715414219
145,145,la'ie,21.6477,-157.9253,22.78,88,8,0.45,US,1715414236
151,151,chipinge,-20.1883,32.6236,26.04,27,0,1.56,ZW,1715414238
161,161,coahuayana de hidalgo,18.7000,-103.6583,21.58,89,2,1.05,MX,1715414242
204,204,belyy yar,53.6039,91.3903,23.94,41,0,1.00,RU,1715414260
241,241,sotouboua,8.5667,0.9833,29.16,66,16,1.55,TG,1715414274
264,264,tabas,33.5959,56.9244,29.12,17,0,1.78,IR,1715414285
278,278,vila velha,-20.3297,-40.2925,23.97,100,0,0.00,BR,1715414186


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

/var/folders/pv/8fr2s7bx25j_xff685sjq2pm0000gn/T/ipykernel_60257/3880923516.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
67,benguela,AO,-12.5763,13.4055,74,
89,namibe,AO,-15.1961,12.1522,71,
100,ardakan,IR,32.3100,54.0175,17,
145,la'ie,US,21.6477,-157.9253,88,
151,chipinge,ZW,-20.1883,32.6236,27,
161,coahuayana de hidalgo,MX,18.7000,-103.6583,89,
204,belyy yar,RU,53.6039,91.3903,41,
241,sotouboua,TG,8.5667,0.9833,66,
264,tabas,IR,33.5959,56.9244,17,
278,vila velha,BR,-20.3297,-40.2925,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 1
params = {
    'categories':categories,
    'limit':limit,
    'apiKey':geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
benguela - nearest hotel: Hotel Moibela
namibe - nearest hotel: Nelsal Pensao
ardakan - nearest hotel: هتل رستوران باغ سرای اردکان
la'ie - nearest hotel: No hotel found
chipinge - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
belyy yar - nearest hotel: No hotel found
sotouboua - nearest hotel: Verger
tabas - nearest hotel: هتل امیر
vila velha - nearest hotel: Hotel Prainha
mahenge - nearest hotel: No hotel found
miguel bocanegra - nearest hotel: Hotel el Sol
rampur - nearest hotel: تمنا ہوٹل اور ریستوران
catanzaro - nearest hotel: Hotel Benny


,City,Country,Lat,Lng,Humidity,Hotel Name
67,benguela,AO,-12.5763,13.4055,74,Hotel Moibela
89,namibe,AO,-15.1961,12.1522,71,Nelsal Pensao
100,ardakan,IR,32.3100,54.0175,17,هتل رستوران باغ سرای اردکان
145,la'ie,US,21.6477,-157.9253,88,No hotel found
151,chipinge,ZW,-20.1883,32.6236,27,No hotel found
161,coahuayana de hidalgo,MX,18.7000,-103.6583,89,No hotel found
204,belyy yar,RU,53.6039,91.3903,41,No hotel found
241,sotouboua,TG,8.5667,0.9833,66,Verger
264,tabas,IR,33.5959,56.9244,17,هتل امیر
278,vila velha,BR,-20.3297,-40.2925,100,Hotel Prainha


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points( 'Lng',
                                    'Lat',
                                    geo=True,
                                    tiles='OSM',
                                    frame_width=800,
                                    frame_height=600,
                                    color='City',
                                    alpha=0.5,
                                    xlabel='x',
                                    ylabel='y',
                                    hover_cols=['Humidity', 'Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)